In [22]:
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn.linear_model import LinearRegression

In [2]:
data = [
    {'luas':1000, 'harga':1000, 'kota':'Jakarta'},
    {'luas':2000, 'harga':2000, 'kota':'Jakarta'},
    {'luas':3000, 'harga':3000, 'kota':'Jakarta'},
    {'luas':4000, 'harga':4000, 'kota':'Jakarta'},
    {'luas':5000, 'harga':5000, 'kota':'Jakarta'},
    {'luas':1000, 'harga':500, 'kota':'Bogor'},
    {'luas':2000, 'harga':1000, 'kota':'Bogor'},
    {'luas':3000, 'harga':1500, 'kota':'Bogor'},
    {'luas':4000, 'harga':2000, 'kota':'Bogor'},
    {'luas':5000, 'harga':2500, 'kota':'Bogor'},
    {'luas':1000, 'harga':2000, 'kota':'Tangsel'},
    {'luas':2000, 'harga':4000, 'kota':'Tangsel'},
    {'luas':3000, 'harga':6000, 'kota':'Tangsel'},
    {'luas':4000, 'harga':8000, 'kota':'Tangsel'},
    {'luas':5000, 'harga':10000, 'kota':'Tangsel'}
]
df = pd.DataFrame(data)
df.corr()

,luas,harga
luas,1.00000,0.62361
harga,0.62361,1.00000


<hr>

## 1. Dummy Var Pandas


In [3]:

dfDummy = pd.get_dummies(df['kota'])
dfDummy


,Bogor,Jakarta,Tangsel
0,0,1,0
1,0,1,0
2,0,1,0
3,0,1,0
4,0,1,0
5,1,0,0
6,1,0,0
7,1,0,0
8,1,0,0
9,1,0,0


In [4]:
dfNew = pd.concat([df,dfDummy], axis = 'columns')
dfNew = dfNew.drop(['kota'],axis = 1)
dfNew.corr()

,luas,harga,Bogor,Jakarta,Tangsel
luas,1.000000e+00,0.623610,1.136868e-17,1.136868e-17,0.000000
harga,6.236096e-01,1.000000,-5.345225e-01,-1.336306e-01,0.668153
Bogor,1.136868e-17,-0.534522,1.000000e+00,-5.000000e-01,-0.500000
Jakarta,1.136868e-17,-0.133631,-5.000000e-01,1.000000e+00,-0.500000
Tangsel,0.000000e+00,0.668153,-5.000000e-01,-5.000000e-01,1.000000


In [5]:
model = linear_model.LinearRegression()

model.fit(dfNew[['luas','Bogor','Tangsel']],dfNew['harga'])

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [6]:
model.coef_
model.intercept_

-500.0

$ y = -500 + 1.16 x luas- 1500 x bogor + 3000 x Tangsel $ 

In [7]:
harga_pred = model.predict(dfNew[['luas','Bogor','Tangsel']])
df['harga_pred'] = harga_pred
df

,luas,harga,kota,harga_pred
0,1000,1000,Jakarta,666.666667
1,2000,2000,Jakarta,1833.333333
2,3000,3000,Jakarta,3000.000000
3,4000,4000,Jakarta,4166.666667
4,5000,5000,Jakarta,5333.333333
5,1000,500,Bogor,-833.333333
6,2000,1000,Bogor,333.333333
7,3000,1500,Bogor,1500.000000
8,4000,2000,Bogor,2666.666667
9,5000,2500,Bogor,3833.333333


<hr>

## 2. Label Encoder Sklearn

In [15]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

label = LabelEncoder()
dfUji = df
dfUji['kota'] = label.fit_transform(dfUji['kota'])
# dfUji
print(label.classes_)
# print(label.transform)

[0 1 2]


In [19]:
x = dfUji[['kota','luas']]
y = dfUji['harga']
# y

In [20]:
ct = ColumnTransformer(
    [('one_hot_encoder',OneHotEncoder(categories='auto'),[0])],
    remainder='passthrough'
)
x = np.array(ct.fit_transform(x), dtype=np.int64)

In [25]:
model1 = LinearRegression()
model1.fit(x,y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [26]:
#[Bogor,Jakarta,Tangsel,luas]
model1.predict([[0,1,0,1000]])

array([666.66666667])

In [27]:
model1.predict(x)

array([ 666.66666667, 1833.33333333, 3000.        , 4166.66666667,
       5333.33333333, -833.33333333,  333.33333333, 1500.        ,
       2666.66666667, 3833.33333333, 3666.66666667, 4833.33333333,
       6000.        , 7166.66666667, 8333.33333333])

In [28]:
df['harga_pred'] = model1.predict(x)
df

,luas,harga,kota,harga_pred
0,1000,1000,1,666.666667
1,2000,2000,1,1833.333333
2,3000,3000,1,3000.000000
3,4000,4000,1,4166.666667
4,5000,5000,1,5333.333333
5,1000,500,0,-833.333333
6,2000,1000,0,333.333333
7,3000,1500,0,1500.000000
8,4000,2000,0,2666.666667
9,5000,2500,0,3833.333333
